In [1]:
import os
import pickle

import torch as T
import yaml
from tqdm import tqdm
from icecream import ic

import sys

sys.path.append('/Users/marcelloceresini/github/FSOD_CenterNet/src')
from data_pipeline import DatasetsGenerator
from model import Model
from evaluation import Evaluate


def load_settings(settings_path: str):
    with open(settings_path, 'r') as f:
        return yaml.safe_load(f)
    
settings = '../settings/model_testing_debug_check.yaml'
    
config = load_settings(settings)

os.makedirs(config['training']['save_training_info_dir'], exist_ok=True)

debug_mode = config['debug']['debug_mode_active']
device = config['device']

# Dataset generator. Only one of these has to be instantiated. It always returns
dataset_gen = DatasetsGenerator(config)


# Instantiate the model (only the base part)
model = Model(config, n_base_classes=len(dataset_gen.train_base.cats))

model = model.to(device)
model.load_state_dict(T.load('../../data/weights/from_server/best_model_fix_stoic.pt', map_location="cpu"))

# Use the dataset generator to generate the base set
dataset_base_train, dataset_base_val, dataset_base_test = dataset_gen.get_base_sets_dataloaders(
    config['training']['batch_size'], config['training']['num_workers'],
    config['training']['pin_memory'], config['training']['drop_last'], 
    shuffle=True
)

print("Before Evaluate")

metrics_test = Evaluate(
            model, 
            dataset_base_test, 
            prefix="test/",
            device=device,
            config=config
        )(is_novel=False)

print("After Evaluate")

with open(os.path.join(config['training']['save_training_info_dir'], 
                        config['training']['base_stats_save_name']), 'wb') as f:
    pickle.dump(metrics_test, f)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=2.06s)
creating index...
index created!
Skipping instantating novel head
Before Evaluate


/Users/marcelloceresini/.pyenv/versions/3.11.4/envs/env-FSOD_CenterNet/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


After Evaluate


In [2]:
ic(metrics_test)

ic| metrics_test: {'test/classes': tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                           14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                           28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                           42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                           56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                           70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                           84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                           98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                          112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                          126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
                       

{'test/map': tensor(0.0002),
 'test/map_50': tensor(0.0010),
 'test/map_75': tensor(0.),
 'test/map_small': tensor(6.9481e-05),
 'test/map_medium': tensor(0.0011),
 'test/map_large': tensor(0.),
 'test/mar_1': tensor(0.0016),
 'test/mar_10': tensor(0.0051),
 'test/mar_100': tensor(0.0056),
 'test/mar_small': tensor(0.0012),
 'test/mar_medium': tensor(0.0098),
 'test/mar_large': tensor(0.),
 'test/map_per_class': tensor([-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00,  0.0000e+00, -1.0000e+00,
          8.0000e-03, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          8.1683e-04, -1.0000e+00,  0.0000e+00, -1.0000e+00,  0.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00,  0.0000e+00,  0.0000e+00,
         -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
         -1.0000e+00,  0.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          0.0000e+00, -1.0000e+00, -1.0000e+00,  0.0000e+00, -1.000